In [ ]:
from dolfin import *
from ufl import le
import operator
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm

import os
if not os.path.exists("./Experiment3-Plots"):
    os.makedirs("./Experiment3-Plots")

params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)

In [ ]:
# Q-Tensor parameters
n = 2
num_steps = 200
T = 2.0
dt = T/num_steps
a = -0.3
b = -4.
c = 4.
M = 1.
L = 1.

# Parameters for electric field
R = 2.
eps2 = 0.5
eps3 = 0.01
eps1 = 2.5

# Parameters for convex splitting
beta1 = 8.
beta2 = 8.

In [ ]:
# Create mesh
mesh = RectangleMesh(Point(-0.5, -0.5), Point(0.5, 0.5), 30, 30)

In [ ]:
# some functions for the scheme
I = Constant(np.eye(n)) # identity matrix
Zero = 0 * I
Zero_func = 0*Function(FunctionSpace(mesh, "Lagrange", 1))

def dF1_dQ(Q):
    Q2 = Q*Q
    trQ2 = tr(Q2)
    result = beta1 * Q - b*(Q2 - 1/n * trQ2 * I) + beta2 * trQ2 * Q
    return result

def dF2_dQ(Q):
    Q2 = Q*Q
    trQ2 = tr(Q2)
    result = (beta1 - a) * Q + (beta2 - c) * trQ2 * Q
    return result

def F_tot(Q):
    Q2 = Q*Q
    trQ2 = tr(Q2)
    trQ3 = tr(Q2*Q)
    result = a/2 * trQ2 - b/3 * trQ3 + c/4 * trQ2**2
    return result

def tens(Q):
    """
    Make vector function into tensor
    """
    return as_tensor([[Q[0], Q[1]],
                      [Q[1], -1*Q[0]]])

def tens_hi(Q):
    return as_tensor([[Q[0], Q[1]],
                      [Zero_func, Zero_func]])

def tens_lo(Q):
    return as_tensor([[Zero_func, Zero_func],
                      [Q[0], Q[1]]])

# approximate Heaviside function
steepness = 5

def H_(x):
    return 1/np.pi * atan(steepness * x) + 0.5

def DH_(x):
    return 1/np.pi * steepness / (1 + (steepness * x)**2)

# truncation of one component to R/d, d=2
def TR_1D(x):
    return x * H_(R/2-x) * H_(R/2+x) + R/2*H_(x-R/2) - R/2*H_(-x-R/2)

# derivative of TR_1D
def calP_1D(x):
    return H_(R/2-x)*H_(R/2+x) - x*DH_(R/2-x)*H_(R/2+x) + x*H_(R/2-x)*DH_(R/2+x) + R/2*DH_(x-R/2) + R/2*DH_(-x-R/2)

# now if Q is a tensor
def TR(Q):
    return as_tensor([[TR_1D(Q[0, 0]), TR_1D(Q[0, 1])], 
                      [TR_1D(Q[1, 0]), TR_1D(Q[1, 1])]])

def calP(Q):
    return as_tensor([[calP_1D(Q[0, 0]), calP_1D(Q[0, 1])], 
                      [calP_1D(Q[1, 0]), calP_1D(Q[1, 1])]])

def calP_h(Q1, Q0):
    P11 = calP_1D(Q0[0, 0])
    P12 = calP_1D(Q0[0, 1])
    P21 = calP_1D(Q0[1, 0])
    P22 = calP_1D(Q0[1, 1])
    
    res = as_tensor([[P11, P12],
                      [P21, P22]])
    return res

def element_mult(A, B):
    return as_tensor([[A[0, 0] * B[0, 0], A[0, 1] * B[0, 1]], 
                      [A[1, 0] * B[1, 0], A[1, 1] * B[1, 1]]])

## Initialize $Q_h^0$ and $u_h^0$

In [ ]:
# initial conditions for the Q tensor
class InitialConditions(UserExpression):
    def eval(self, values, x):
        x, y = x[0], x[1]
        if np.isclose(x, -0.5) or np.isclose(x, 0.5) or np.isclose(y, -0.5) or np.isclose(y, 0.5):
            values[0] = -0.5
            values[1] = 0.0
        else:
            n0 = 0
            n1 = 1
            d = np.array([[n0], [n1]])
            Q0 = d @ d.T - np.sum(d*d)/2.0 * np.eye(2) # Q tensor

            values[0] = Q0[0, 0]
            values[1] = Q0[0, 1]
    def value_shape(self):
        return (2,)

# Define and solve scheme

In [ ]:
class QTensorEquation(NonlinearProblem):
    def __init__(self, L, a, bcs):
        NonlinearProblem.__init__(self)
        self.L = L
        self.a = a
        self.bcs = bcs
    def F(self, b, x):
        assemble(self.L, tensor=b)
        for bc in self.bcs:
            bc.apply(b,x)
    def J(self, A, x):
        assemble(self.a, tensor=A)
        for bc in self.bcs:
            bc.apply(A)

In [ ]:
def simulate(ELECTRIC_FIELD_STRENGTH=1.0):

    # initialize Q tensor at time 0
    Q0 = Function(VectorFunctionSpace(mesh, "Lagrange", 1, dim=2))
    Q_init = InitialConditions()
    Q0.interpolate(Q_init)

    # define uninitialized u0 and linear form to solve for u0
    u_space = FunctionSpace(mesh, "Lagrange", 1)
    u0 = Function(u_space)
    v = TestFunction(u_space)

    L_elliptic = inner(grad(u0) + eps2*TR(tens(Q0))*grad(u0) + eps3*div(tens(Q0)), grad(v))*dx

    # define boundary condition for u0
    t = 0.0
    u_b = Expression(f'{0}*x[0]', degree=2)
    bc_u = DirichletBC(u_space, u_b, DomainBoundary())

    # solve for u0
    solve(L_elliptic == 0, u0, bc_u)

    Q_space = VectorElement("Lagrange", mesh.ufl_cell(), 1, dim=2)
    u_space = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
    ME = MixedElement(Q_space, u_space)
    V = FunctionSpace(mesh, ME)

    # Define trial and test functions
    dQu = TrialFunction(V)
    q, v = TestFunctions(V)
    r, _ = TestFunctions(V)

    # Define functions
    Qu = Function(V)

    # Split mixed functions
    dQ, du = split(dQu)
    Q, u = split(Qu)

    # Weak statement of the equation
    L_form = inner(tens(Q - Q0), tens(q))*dx\
        + M*L*dt/2*inner(grad(tens(Q+Q0)), grad(tens(q)))*dx\
        + M*dt*inner(dF1_dQ(tens(Q)), tens(q))*dx\
        - M*dt*inner(dF2_dQ(tens(Q0)), tens(q))*dx\
        - M*eps2*dt/2*(inner(element_mult(calP_h(tens(Q), tens(Q0)), outer(grad(u0), grad(u))) - (1/n)*tr(element_mult(calP_h(tens(Q), tens(Q0)), outer(grad(u0), grad(u))))*I, tens(q))*dx)\
        - M*eps3*dt/2*(inner(grad(u+u0), div(sym(tens_hi(q))))*dx - (1/n)*inner(grad(u+u0), grad(tr(tens(q))))*dx)\
        + dt*inner(grad(u) + eps2*TR(tens(Q))*grad(u) + eps3*div(tens(Q)), grad(v))*dx
    
    a_form = derivative(L_form, Qu, dQu)

    # boundary conditions for Q do not change
    # ( 0 ) ( 0  1 )     ( 0  0 ) subtract  ( -0.5  0 )
    # (   )           =  (      )   ==>     (         )
    # ( 1 )              ( 0  1 ) trace/2   ( 0   0.5 )
    bc_Q = DirichletBC(V.sub(0), np.array([-0.5, 0.0]), DomainBoundary())

    # Store solutions and times
    Q_array = Q0.vector()[:].reshape(-1, 2)
    Q_11 = Q_array[:, 0]
    Q_12 = Q_array[:, 1]
    solutions_Q = [np.array([[Q_11, Q_12], [Q_12, -Q_11]]).transpose(2, 0, 1)]
    solutions_u = [np.array(u0.vector()[:].reshape(-1, 1))]
    times = [0.0]

    solver = NewtonSolver()

    t = 0.0
    while (t < T):
        print(t)
        t += dt
        times.append(t)
        
        if t >= 0.75*T:
            u_b = Expression(f'{ELECTRIC_FIELD_STRENGTH}*x[0]', degree=2)
        elif t >= 0.5*T:
            u_b = Expression(f'{2/3*ELECTRIC_FIELD_STRENGTH}*x[0]', degree=2)
        elif t >= 0.25*T:
            u_b = Expression(f'{1/3*ELECTRIC_FIELD_STRENGTH}*x[0]', degree=2)
        else:
            u_b = Expression(f'{0}*x[0]', degree=2)
        bc_u = DirichletBC(V.sub(1), u_b, DomainBoundary())

        problem = QTensorEquation(L_form, a_form, [bc_Q, bc_u])
        solver.solve(problem, Qu.vector())

        Q_array = Qu.sub(0, deepcopy=True).vector()[:].reshape(-1, 2)
        Q_11 = Q_array[:, 0]
        Q_12 = Q_array[:, 1]

        solutions_Q.append(np.array([[Q_11, Q_12], [Q_12, -Q_11]]).transpose(2, 0, 1))
        solutions_u.append(np.array(Qu.sub(1, deepcopy=True).vector()[:].reshape(-1, 1)))

        Q0.vector()[:] = np.ascontiguousarray(Qu.sub(0, deepcopy=True).vector()[:])
        u0.vector()[:] = np.ascontiguousarray(Qu.sub(1, deepcopy=True).vector()[:])

    solutions_u = np.array(solutions_u)
    solutions_Q = np.array(solutions_Q) # time x mesh size x rows x cols

    # turn solutions into time series of vector field and use plot command from FEniCS
    directors = np.zeros((solutions_Q.shape[0], solutions_Q.shape[1], 2, 1))
    for time in range(solutions_Q.shape[0]):
        for point in range(solutions_Q.shape[1]):
            Q_tensor = solutions_Q[time, point]
            eigenvalues, eigenvectors = np.linalg.eigh(Q_tensor)
            v1 = eigenvectors[:, 0]
            v2 = eigenvectors[:, 1]
            if np.sum(np.abs(Q_tensor)) > 1e-6:
                director1x = v1[0]
                director1y = v1[1]
                director2x = v2[0]
                director2y = v2[1]
                directors[time, point] = v2.reshape(-1, 1)
            else:
                directors[time, point] = np.zeros((2, 1))

    return times, directors, solutions_u, np.mean(np.abs(np.arctan(directors[-1, :, 1, 0]/directors[-1, :, 0, 0])) / (np.pi/2))

In [ ]:
efs = 10.0
times, directors, solutions_u, fraction = simulate(ELECTRIC_FIELD_STRENGTH=efs)

# Visualize solution

In [ ]:
DirectorSpace = VectorFunctionSpace(mesh, "Lagrange", 1, dim=n)
D0 = Function(DirectorSpace)
u1 = Function(FunctionSpace(mesh, "Lagrange", 1))
for time in range(directors.shape[0]):
    fig, ax = plt.subplots(1, 1)
    
    u1.vector()[:] = np.ascontiguousarray(solutions_u[time].reshape(-1))
    out = plot(u1, vmin=np.min(solutions_u), vmax=np.max(solutions_u))
    
    D0.vector()[:] = np.ascontiguousarray(directors[time].reshape(-1))
    out = plot(D0, scale=25., pivot='mid', clim=(0., 0.), headaxislength=0)
    
    norm = mpl.colors.Normalize(vmin=np.min(solutions_u), vmax=np.max(solutions_u))
    if abs(time*dt - 2) <= 1e-4:
        cbar = ax.figure.colorbar(mpl.cm.ScalarMappable(norm=norm))

    print(f"{times[time]:.2f}")
    if time*dt >= 0.75*T:
        plt.title(f"Extension = ${efs:.2f}\cdot x$")
    elif time*dt >= 0.5*T:
        plt.title(f"Extension = ${2*efs/3:.2f}\cdot x$")
    elif time*dt >= 0.25*T:
        plt.title(f"Extension = ${efs/3:.2f}\cdot x$")
    else:
        plt.title(f"Extension $\equiv 0$")
    plt.savefig(f"./Experiment3-Plots/experiment3_{time}.pdf", bbox_inches='tight')
    plt.savefig(f"./Experiment3-Plots/experiment3_{time}.png", bbox_inches='tight')
    plt.show()

In [ ]:
fraction # average director angle / (pi/2)

In [ ]:
import imageio

with imageio.get_writer('./Experiment3-Plots/Fréedericksz_Progressive.gif', mode='I') as writer:
    for time in range(directors.shape[0]):
        filename = f"./Experiment3-Plots/experiment3_{time}.png"
        image = imageio.imread(filename)
        writer.append_data(image)

In [ ]:
fractions = []
strengths = np.arange(2.0, 15.5, 0.5)
for ELECTRIC_FIELD_STRENGTH in strengths:
    times, directors, solutions_u, fraction = simulate(ELECTRIC_FIELD_STRENGTH)
    fractions.append(fraction)

In [ ]:
plt.plot(strengths, fractions)
plt.ylabel("Average of Director Angle$\cdot(\pi/2)^{-1}$")
plt.xlabel("Electric Field Strength")
plt.savefig("./Experiment3-Plots/Fréedericksz_transition.pdf", bbox_inches='tight')
plt.show()